In [58]:
import math

class Value():

    def __init__(self, num, _parents=(), _op=''):
        self.num = num
        self.grad = 0.0
        self.backProp = lambda: None
        self._parents = _parents
        self._op = _op

    def __repr__(self):
        return f"Value(num:{self.num})"

    def __add__(self, other):

        out = Value(self.num + other.num, (self, other), '+')

        def _backProp():
            other.grad = 1.0 * out.grad
            self.grad = 1.0 * out.grad

        self.backProp = _backProp

        return out

    def __sub__(self, other):
        temp = other.copy(num = other.num * -1)
        return self.add(self, temp)

    def __mul__(self, other):
        out = Value(self.num * other.num, (self, other), '*')

        def _backProp():
            self.grad = other.num * out.grad
            other.grad = self.num * out.grad

        self.backProp = _backProp

        return out    

    def __div__(self, other):
        return Value(self.num / other.num, (self, other), '/')

    def tanh(self):
        numValue = (math.exp(2 * self.num) - 1) / (math.exp(2 * self.num) + 1)
        out = Value(numValue, (self,), 'tanH')

        def _backProp():
            self.grad = (1 - numValue ** 2) * out.grad
    
        self.backProp = _backProp
    
        return out

    def gradient(self):
        for parent in self._parents:
            parent.backProp()
            parent.gradient()

In [59]:
a = Value(1.0)
b = Value(0.5)
w1 = Value(0.3424)
w2 = Value(0.7424)
bias = Value(0.54142)
out1 = w1*a + w2*b + bias
out2 = out1.tanh()
out2.grad = 1.0
out2.gradient()
print(out2.grad)
print(out1.grad)
print(a.grad)
print(b.grad)
print(w1.grad)
print(w2.grad)
print(bias.grad)

1.0
0.27803481988494216
0.0951991223286042
0.20641305028258106
0.27803481988494216
0.13901740994247108
0.27803481988494216


In [100]:
import random

class Neuron():

    def __init__(self, numInp):
        self.weights = [Value(random.uniform(-1, 1)) for _ in range(numInp)]
        self.bias = Value(random.uniform(-1, 1))

    def __call__(self, inputs):
        assert(len(inputs) == len(self.weights))
        out1 = Value(sum([w.num * x.num for w,x in zip(self.weights, inputs)]) + self.bias.num)
        out2 = out1.tanh()
        return out2

class Layer():

    def __init__(self, numNeurons, numInp):
        print(numNeurons)
        self.neurons = [Neuron(numInp) for _ in range(numNeurons)]

    def __call__(self, inputs):
        out = [neuron(inputs) for neuron in self.neurons]
        return out

class MLP():

    def __init__(self, numLayers, numNeuronsPerLayer, numInps):
        self.layers = []
        for i in range(numLayers):
            self.layers.append(Layer(numNeuronsPerLayer[i], numInps))
            numInps = numNeuronsPerLayer[i]

    def __call__(self, inputs):
        for layer in self.layers:
            inputs = layer(inputs)
        return inputs

In [101]:
network = MLP(3, [3, 6, 1], 2)
a = network([Value(1.0), Value(2.0)])
a = network([Value(1.0), Value(2.0)])
a[0].gradient()
print(a)

3
6
1
[Value(num:-0.8710075899414912)]


In [107]:
x = [
    [Value(random.uniform(-1, 1)) for _ in range(3)],
    [Value(random.uniform(-1, 1)) for _ in range(3)],
    [Value(random.uniform(-1, 1)) for _ in range(3)],
    [Value(random.uniform(-1, 1)) for _ in range(3)],
]

y = [Value(random.uniform(-1, 1)) for _ in range(4)]

mlp = MLP(3, [3, 6, 1], 3)
y_pred = [mlp(x_i)[0] for x_i in x]
print(y_pred)
# loss = sum([y1 - y2 for y1, y2 in zip(y_pred, y)])
print(loss)

3
6
1
[Value(num:0.7833312921499127), Value(num:0.688127916915516), Value(num:0.7572835113210662), Value(num:0.4794003841315886)]
-0.5878890943396743
